In [ ]:
#!pip install polaris-ml

In [34]:
import os
import pandas as pd
import numpy as np

DATASET_PATH='datasets-preprocessed/lightsail_dataset.csv'
lista_archivos_lightsail = ["../../src/datasets/lightsail2/" + x for x in os.listdir("../../src/datasets/lightsail2/") if x[-3:] == "txt"]

#lista_archivos_marsexpress = ["./mars_express/" + x for x in os.listdir("./mars_express/") if x[-3:] == "csv"]
print(lista_archivos_lightsail[:5])
#print(lista_archivos_marsexpress[:5])

['C:/Users/manni/Desktop/VIU/TFM/repo/Improve-MBSE-ML/src/datasets/lightsail2/2127458.txt', 'C:/Users/manni/Desktop/VIU/TFM/repo/Improve-MBSE-ML/src/datasets/lightsail2/2127459.txt', 'C:/Users/manni/Desktop/VIU/TFM/repo/Improve-MBSE-ML/src/datasets/lightsail2/2127460.txt', 'C:/Users/manni/Desktop/VIU/TFM/repo/Improve-MBSE-ML/src/datasets/lightsail2/2127461.txt', 'C:/Users/manni/Desktop/VIU/TFM/repo/Improve-MBSE-ML/src/datasets/lightsail2/2127462.txt']


In [ ]:
#DATASET PARA LECTURA HUMANA DEL LIGHTSAIL

def read_human_from_txt_lightsail(lista_archivos):

  etiquetas = ["Observed_at", "Posted_at"]

  dataset = {}

  dataset[etiquetas[0]] = []
  dataset[etiquetas[1]] = []

  archivo1 = open(lista_archivos[0], "r")

  for linea in archivo1:
    if "Observed at" in linea:
      valor = linea.split("at:")[1].strip()
      dataset["Observed_at"].append(valor)
    elif "Posted at" in linea:
      valor = linea.split("at:")[1].strip()
      dataset["Posted_at"].append(valor)
    elif ":" in linea and "=" in linea:
      etiqueta = linea.split(":")[1].split("=")[0].strip().replace(" ", "_").replace("[", "_").replace("]", "_").replace("<", "_")
      valor = linea.split(":")[1].split("=")[1].split("[")[0].strip()
      dataset[etiqueta] = []
      dataset[etiqueta].append(valor)

  for nombre_archivo in lista_archivos[1:]:
    archivo = open(nombre_archivo, "r")
    for linea in archivo:
      if "Observed at" in linea:
        valor = linea.split("at:")[1].strip()
        dataset["Observed_at"].append(valor)
      elif "Posted at" in linea:
        valor = linea.split("at:")[1].strip()
        dataset["Posted_at"].append(valor)
      elif ":" in linea and "=" in linea:
        etiqueta = linea.split(":")[1].split("=")[0].strip().replace(" ", "_").replace("[", "_").replace("]", "_").replace("<", "_")
        valor = linea.split(":")[1].split("=")[1].split("[")[0].strip()
        dataset[etiqueta].append(valor)

  df = pd.DataFrame(dataset.values(), dataset.keys())
  df = df.transpose()
  return df


In [63]:
#LECTURA DE LOS DATASETS DE LIGHTSAIL Y MARS EXPRESS
from datetime import datetime

def transform_unix_to_utc(series):
  if len(series) > 0:
    new_series = [str(datetime.utcfromtimestamp(x/1000)) for x in series]
    return new_series
  else:
    return []

'''def transform_utc_to_unix(series):
    dt = datetime.datetime.strptime('20170124T1815', '%Y%m%dT%H%M')
  if len(series) > 0:
    new_series = [str(datetime.utcfromtimestamp(x/1000)) for x in series]
    return new_series
  else:
    return []
'''
def read_from_txt_lightsail(lista_archivos):
  
  dataframeTotal = pd.DataFrame()
  dataframeN = pd.DataFrame(columns=["time"])
  for nombre_archivo in lista_archivos:
    archivo = open(nombre_archivo, "r")
    kvp_form = False
    for linea in archivo:
      if "KVP form:" in linea:
        kvp_form = True
        dictN = {}
      if kvp_form is True and "=" in linea:
        etiqueta = linea.split("=")[0].strip()
        valor = float(linea.split("=")[1].split(",")[0].strip())
        dictN[etiqueta] = valor
    df_toappend = pd.DataFrame([dictN])
    dataframeN = dataframeN.append(df_toappend, ignore_index=True)
    dataframeN.sort_values("time")
  #df = pd.DataFrame(dataset.values(), dataset.keys())
  #df = df.transpose()
  #dataframeN.sort_values("time")
  return dataframeN


def read_from_ut_ms(lista_archivos):
  columnas = []
  for archivo in lista_archivos:
    df1 = pd.read_csv(archivo)
    columnas += list(df1.columns)
    columnas = list(set(columnas))

  df = pd.DataFrame(columns=columnas)
  df = df.loc[:, ~df.columns.duplicated()]
  for archivo in lista_archivos:
    df2 = pd.read_csv(archivo)
    df2["date"] = transform_unix_to_utc(df2["ut_ms"])
    df = df.append(df2)
  return df

def read_from_utb_ms(lista_archivos):
  columnas = []
  for archivo in lista_archivos:
    df1 = pd.read_csv(archivo)
    columnas = list(df1.columns)
    columnas.append("date")
  df = pd.DataFrame(columns=columnas)
  for archivo in lista_archivos:
    df2 = pd.read_csv(archivo)
    df2["date"] = transform_unix_to_utc(df2["utb_ms"])
    df = df.append(df2)
  return df

def read_from_csv_mars_express(lista_archivos_marsexpress):

  lista_archivos_saaf = [x for x in lista_archivos_marsexpress if x.split("--")[-1].split(".")[0] == "saaf"]
  lista_archivos_dmop = [x for x in lista_archivos_marsexpress if x.split("--")[-1].split(".")[0] == "dmop"]
  lista_archivos_ftl = [x for x in lista_archivos_marsexpress if x.split("--")[-1].split(".")[0] == "ftl"]
  lista_archivos_evtf = [x for x in lista_archivos_marsexpress if x.split("--")[-1].split(".")[0] == "evtf"]
  lista_archivos_ltdata = [x for x in lista_archivos_marsexpress if x.split("--")[-1].split(".")[0] == "ltdata"]

  df_saaf = read_from_ut_ms(lista_archivos_saaf)
  df_dmop = read_from_ut_ms(lista_archivos_dmop)
  df_ftl = read_from_utb_ms(lista_archivos_ftl)
  df_evtf = read_from_ut_ms(lista_archivos_evtf)
  df_ltdata = read_from_ut_ms(lista_archivos_ltdata)

  all_columns = list(df_saaf.columns) + list(df_dmop.columns) + list(df_ftl.columns) + list(df_evtf.columns) + list(df_ltdata.columns)
  all_columns = list(set(all_columns))

  df = pd.DataFrame(columns=all_columns)
  df = df.append(df_saaf, ignore_index=True)
  df = df.append(df_dmop, ignore_index=True)
  df = df.append(df_ftl, ignore_index=True)
  df = df.append(df_evtf, ignore_index=True)
  df = df.append(df_ltdata, ignore_index=True).sort_values("date")


  return df

In [64]:
#COMPROBAR RESULTADOS ANTES DE UTILIZAR POLARIS
df = read_from_txt_lightsail(lista_archivos_lightsail)
display(df.head(27))

,time,atmelPwr_curr,atmelPwr_volt,threeV_plTmp,threeV_plPwr_volt,threeV_plPwr_curr,fiveV_plPwr_volt,fiveV_plPwr_curr,daughter_aTmp,daughter_aPwr_volt,...,deploy_armed,pic_on,pic_busy,pic_status,motor_pos,motor_limit,wheel_speed,ss_votes_lt3_wrap,ss_votes_eq3_wrap,ss_votes_gt3_wrap
0,1.562478e+09,2880.0,247808.0,18980.0,217088.0,1536.0,331776.0,2048.0,18980.0,0.0,...,0.0,0.0,0.0,0.0,0.0,135563.0,0.0,NaN,NaN,NaN
1,1.562463e+09,2208.0,331776.0,18084.0,217088.0,1536.0,331776.0,2048.0,18052.0,0.0,...,0.0,0.0,0.0,0.0,0.0,135563.0,0.0,NaN,NaN,NaN
2,1.562463e+09,1312.0,333824.0,18052.0,217088.0,1536.0,331776.0,2048.0,18084.0,0.0,...,0.0,0.0,0.0,0.0,0.0,135563.0,0.0,NaN,NaN,NaN
3,1.562463e+09,2304.0,333824.0,18052.0,217088.0,1536.0,331776.0,2048.0,18052.0,0.0,...,0.0,0.0,0.0,0.0,0.0,135563.0,0.0,NaN,NaN,NaN
4,1.562463e+09,2176.0,333824.0,18052.0,217088.0,1536.0,331776.0,2048.0,18084.0,0.0,...,0.0,0.0,0.0,0.0,0.0,135563.0,0.0,NaN,NaN,NaN
5,1.562463e+09,1376.0,325632.0,18052.0,217088.0,2560.0,331776.0,2048.0,18052.0,0.0,...,0.0,0.0,0.0,0.0,0.0,135563.0,0.0,NaN,NaN,NaN
6,1.562463e+09,1728.0,337920.0,17924.0,217088.0,1536.0,331776.0,2048.0,17892.0,0.0,...,0.0,0.0,0.0,0.0,0.0,135563.0,0.0,NaN,NaN,NaN
7,1.562463e+09,1280.0,335872.0,17892.0,217088.0,1536.0,331776.0,2048.0,17892.0,0.0,...,0.0,0.0,0.0,0.0,0.0,135563.0,0.0,NaN,NaN,NaN
8,1.562463e+09,2272.0,337920.0,17892.0,217088.0,1536.0,331776.0,2048.0,17924.0,0.0,...,0.0,0.0,0.0,0.0,0.0,135563.0,0.0,NaN,NaN,NaN
9,1.562463e+09,2144.0,339968.0,17860.0,217088.0,1536.0,331776.0,2048.0,17892.0,0.0,...,0.0,0.0,0.0,0.0,0.0,135563.0,0.0,NaN,NaN,NaN


In [27]:
df

""


In [65]:
#GUARDAR CSV DEL LIGHTSAIL2 PARA PROCESAMIENTO DE POLARIS
df_csv = df.to_csv(DATASET_PATH, sep=',', index=False)


In [3]:
#GUARDAR CSV DEL MARS EXPRESS PARA PROCESAMIENTO DE POLARIS
df_csv = df.to_csv('marsexpress_preprossed.csv', sep=',', index=False)

NameError: name 'df' is not defined

In [ ]:
#ANALISYS.PY DE POLARIS, PARA PODER PERSONALIZARLO SEGÚN EL DATASET CONVENGA

"""
Module to launch different data analysis.
"""
import logging

from fets.math import TSIntegrale
from mlflow import set_experiment

from polaris.data.graph import PolarisGraph
from polaris.data.readers import read_polaris_data
from polaris.dataset.metadata import PolarisMetadata
from polaris.learn.feature.extraction import create_list_of_transformers, \
    extract_best_features
from polaris.learn.predictor.cross_correlation import XCorr
from polaris.learn.predictor.cross_correlation_configurator import \
    CrossCorrelationConfigurator

LOGGER = logging.getLogger(__name__)


class NoFramesInInputFile(Exception):
    """Raised when frames dataframe is empty"""


def feature_extraction(input_file, param_col):
    """
    Start feature extraction using the given settings.

        :param input_file: Path of a CSV file that will be
            converted to a dataframe
        :type input_file: str
        :param param_col: Target column name
        :type param_col: str
    """
    # Create a small list of two transformers which will generate two
    # different pipelines
    transformers = create_list_of_transformers(["5min", "15min"], TSIntegrale)

    # Extract the best features of the two pipelines
    out = extract_best_features(input_file,
                                transformers,
                                target_column=param_col,
                                time_unit="ms")

    # out[0] is the FeatureImportanceOptimization object
    # from polaris.learn.feature.selection
    # pylint: disable=E1101
    print(out[0].best_features)


# pylint: disable-msg=too-many-arguments
def cross_correlate(input_file,
                    output_graph_file=None,
                    xcorr_configuration_file=None,
                    graph_link_threshold=0.1,
                    use_gridsearch=False,
                    csv_sep=',',
                    force_cpu=False):
    """
    Catch linear and non-linear correlations between all columns of the
    input data.

        :param input_file: CSV or JSON file path that will be
            converted to a dataframe
        :type input_file: str
        :param output_graph_file: Output file path for the generated graph.
            It will overwrite if the file already exists. Defaults to None,
            which is'/tmp/polaris_graph.json'
        :type output_graph_file: str, optional
        :param xcorr_configuration_file: XCorr configuration file path,
            defaults to None. Refer to CrossCorrelationConfigurator for
            the default parameters
        :type xcorr_configuration_file: str, optional
        :param graph_link_threshold: Minimum link value to be considered
            as a link between two nodes
        :type graph_link_threshold: float, optional
        :param use_gridsearch: Use grid search for the cross correlation.
            If this is set to False, then it will just use regression.
            Defaults to False
        :type use_gridsearch: bool, optional
        :param csv_sep: The character that separates the columns inside of
            the CSV file, defaults to ','
        :type csv_sep: str, optional
        :param force_cpu: Force CPU for cross corelation, defaults to False
        :type force_cpu: bool, optional
        :raises NoFramesInInputFile: If there are no frames in the converted
            dataframe
    """
    # Reading input file - index is considered on first column
    metadata, dataframe = read_polaris_data(input_file, csv_sep)

    if dataframe.empty:
        LOGGER.error("Empty list of frames -- nothing to learn from!")
        raise NoFramesInInputFile

    input_data = normalize_dataframe(dataframe)
    source = metadata['satellite_name']

    set_experiment(experiment_name=source)

    xcorr_configurator = CrossCorrelationConfigurator(
        xcorr_configuration_file=xcorr_configuration_file,
        use_gridsearch=use_gridsearch,
        force_cpu=force_cpu)

    # Creating and fitting cross-correlator
    xcorr = XCorr(metadata, xcorr_configurator.get_configuration())
    xcorr.fit(input_data)

    if output_graph_file is None:
        output_graph_file = "/tmp/polaris_graph.json"

    metadata = PolarisMetadata({"satellite_name": source})
    graph = PolarisGraph(metadata=metadata)
    graph.from_heatmap(xcorr.importances_map, graph_link_threshold)
    with open(output_graph_file, 'w') as graph_file:
        graph_file.write(graph.to_json())


def normalize_dataframe(dataframe):
    """
        Apply dataframe modification so it's compatible
        with the learn module. The time column is first
        set as the index of the dataframe. Then, we drop
        the time column.

        :param dataframe: The pandas dataframe to normalize
        :type dataframe: pd.DataFrame
        :return: Pandas dataframe normalized
        :rtype: pd.DataFrame
    """
    dataframe.index = dataframe.date
    dataframe.drop(['date'], axis=1, inplace=True)

    return dataframe


In [ ]:
#GENERA EL GRAFO DE LIGHTSAIL2
cross_correlate("lightsail_preprossed.csv")


In [ ]:
#GENERA EL GRAFO DE MARS EXPRESS
cross_correlate("marsexpress_preprossed.csv")